In [43]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

from sklearn.preprocessing import FunctionTransformer
from sentence_transformers import SentenceTransformer
from sklearn.svm import SVC

from sklearn.metrics import f1_score

from sklearn.model_selection import GridSearchCV

In [44]:
data = pd.read_csv('data/train.csv')
data.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [45]:
transformer = SentenceTransformer('all-mpnet-base-v2')

In [46]:
sklearn_transformer = FunctionTransformer(lambda x: transformer.encode(x, convert_to_tensor=True, show_progress_bar=False).detach().cpu().numpy())

In [47]:
pipeline = Pipeline(
    [
        ("clf", SVC()),
    ]
)
pipeline

Pipeline(steps=[('clf', SVC())])

In [62]:
param_grid = {
    "clf__kernel": ('linear', 'poly', 'rbf', 'sigmoid'),
    "clf__C": (0.00001, 0.0001, 0.001, 0.01, 1, 10, 100, 1000)
}

In [63]:
grid = GridSearchCV(pipeline, param_grid, scoring='f1')

In [50]:
X_train, X_test, y_train, y_test = train_test_split(
    sklearn_transformer.transform(data["text"]), 
    data["target"]
)

In [64]:
grid.fit(X_train, y_train)

GridSearchCV(estimator=Pipeline(steps=[('clf', SVC())]),
             param_grid={'clf__C': (1e-05, 0.0001, 0.001, 0.01, 1, 10, 100,
                                    1000),
                         'clf__kernel': ('linear', 'poly', 'rbf', 'sigmoid')},
             scoring='f1')

In [65]:
y_predicted = grid.predict(X_test)

In [66]:
f1_score(y_test, y_predicted)

0.7961414790996785

In [67]:
grid.best_score_, grid.best_params_, grid.best_estimator_

(0.7943619513908937,
 {'clf__C': 1, 'clf__kernel': 'rbf'},
 Pipeline(steps=[('clf', SVC(C=1))]))

In [68]:
test_data = pd.read_csv('data/test.csv')

In [69]:
target = grid.predict(sklearn_transformer.transform(test_data["text"]))

In [70]:
output = pd.DataFrame(data = {
    "id": test_data["id"],
    "target": target
})
output.head()

,id,target
0,0,0
1,2,1
2,3,1
3,9,1
4,11,1


In [72]:
output.to_csv("submission.csv", index=False)

In [61]:
!kaggle competitions submit -c nlp-getting-started -f "submission.csv" -m "Submission using simple SVC and Sbert vectorizer with grid search params"

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
zsh:1: command not found: kaggle


In [41]:
!pip3 install kaggle

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Defaulting to user installation because normal site-packages is not writeable
